#Подготовка

##Импорт и установка библиотек

In [123]:
# Установка pydicom
%pip install pydicom

Note: you may need to restart the kernel to use updated packages.


In [124]:
%pip install nibabel

Note: you may need to restart the kernel to use updated packages.


In [125]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [126]:
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [127]:
# Модель
from tensorflow.keras import Model, Input, regularizers

# Загрузка модели
from tensorflow.keras.models import load_model

# Слои
from tensorflow.keras.layers import Conv3D, BatchNormalization, Activation, Add
from tensorflow.keras.layers import MaxPooling3D, Conv3DTranspose, concatenate

# Оптимизаторы
from keras.optimizers import Adam

# Метрики
from tensorflow.keras.metrics import BinaryIoU

# Регуляризация параметров модели
from tensorflow.keras.regularizers import l2

# Схема модели
from tensorflow.keras.utils import plot_model

#Для сохранения и загрузки модели с кастомными метриками
#from keras.utils.generic_utils import get_custom_objects

# Для низкоуровневого доступа к операциям и тензорам
import tensorflow.keras.backend as K

# Для работы с изображениями
from PIL import Image

# Для работы с dcm-изображениями
import pydicom as dicom

# Для работы с nii-изображениями
import nibabel as nib

# Инструменты для работы с массивами
import numpy as np

# Для уменьшения размеров изображений в numpy
from scipy.ndimage import zoom

# Для работы с файлами
import os, shutil

# Для работы с датой/временем
import datetime

# очистка ОЗУ
import gc

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

##Глобальные параметры

In [128]:
CLASS_COUNT = 1                                             # Количество классов (с выпотом и без, бинарная сегментация)
MIN_BOUND = 0                                               # Минимальная граница расположения выпота
MAX_BOUND = 400                                             # Максимальная граница расположения выпота
MAX_SLICES = 304                                            # Максимальное количество срезов
PROJECT_DIR = 'D:\\Projects\\AIU\\2_Radlogics'              # Папка с проектом
DATA_DIR = os.path.join(PROJECT_DIR, 'datasets')            # Папка с данными
LEARNING_DIR = os.path.join(PROJECT_DIR, 'learning')        # Чекпоинты
MODEL = 0                                                   # Модель: 0 - U-Net 3D, 1 - VoxResNet
IMG_RESOLUTION = 512                                        # Разрешение среза
BATCH_SIZE = 1                                              # Размер батча
SIZE_STEP = BATCH_SIZE                                      # Шаг смещения
LAST_EPOCH = 0                                              # Последняя эпоха, применяется при дообучении модели

# Управляющие флаги
CLASS_EQUATING_f = True                                     # Уравнивание классов
VALIDATION_f = True                                         # Использование проверочной выборки при обучении
FULL_DATASET_f = True                                       # Полный датасет или частичный
PNG_MASKS_f = True                                          # Использование сегментированных 2D изображений в формате png вместо 3D в nii.gz
SIMPLE_3D_f = False                                         # Переключение полной и упрощённой 3D сегментации // не реализовано
AUGMENTATION_f = False                                      # Флаг применения аугментации // не реализовано
ADDITIONAL_TRAIN_f = False                                  # Режим дообучения, с загрузкой весов

# Пути к папкам с оригинальными изображениями и плевральными масками
original_dir = os.path.join(DATA_DIR, 'Original')   # Папка с оригинальными DICOM файлами
effusions_dir = os.path.join(DATA_DIR, 'Effusions')         # Папка с файлами плевральных выпотов
thor_cav_dir = os.path.join(DATA_DIR, 'Thoracic_Cavities')  # Папка с файлами грудных полостей

##Полезные функции


*   get_full_path(sub_path)
*   get_folders(path)
*   plot_pics(imgs, titles)
*   Model checkpoints

In [129]:
# Функция получения пути к папке с изображениями, будь то:
# LUNG1-XXX;
# LUNG1-XXX/какая-то_папка
# LUNG1-XXX/какая-то_папка1/какая-то_папка2

def get_full_path(sub_path):  # sub_path - полный путь к папке LUNG1-XXX

    full_path = sub_path
    for _ in range(2):
        if len(os.listdir(full_path)) > 0:
            if os.path.exists(os.path.isdir(os.path.join(full_path, os.listdir(full_path)[0]))):
                path = os.path.join(full_path, os.listdir(full_path)[0])
                if os.path.isdir(path): full_path = path

    return full_path

In [130]:
# Функция получения списка исключительно папок в определённой директории

def get_folders(path):                        # путь к папке с данными
    # Создаём список, содержащий только папки
    folders = []
    for item in os.listdir(path):             # список элементов в директории
        full_path = os.path.join(path, item)
        if os.path.isdir(full_path):
            folders.append(item)

    return folders

In [131]:
# Функция для отображения изображений

def plot_pics(imgs, titles):

    num = len(imgs)
    plt.figure(1, figsize=(3*num, 3))

    for i in range(num):
        plt.subplot(1, num, i+1)
        plt.title(titles[i])
        plt.imshow(imgs[i], cmap='gray')
        plt.axis('off')

    plt.show()

In [132]:
# Функция для сохранения директорий данных и масок выборок в файл

def save_parameters(model_name, train, val, test):

    with open(os.path.join(LEARNING_DIR, f'{model_name}_parameters.txt'), mode="a") as file:
        # получаем текущее время
        current_time = datetime.datetime.now()
        
        # записываем дату и время
        file.write(f'Произведено: {current_time}')

        # записываем наименования директорий
        file.write(f'\nДиректории оригинальных изображений:\n')
        file.write(' '.join(map(str, orig_dir_np)))
        file.write(f'\nДиректории масок с выпотами:\n')
        file.write(' '.join(map(str, eff_dir_np)))

        # записываем маски
        file.write(f'\nМаска для обучающей выборки:\n')
        file.write(' '.join(map(str, train.astype(np.byte))))
        file.write(f'\nМаска для проверочной выборки:\n')
        file.write(' '.join(map(str, val.astype(np.byte))))
        file.write(f'\nМаска для тестовой выборки:\n')
        file.write(' '.join(map(str, test.astype(np.byte))) + '\n')


# Функции сохранения весов и полной модели

def save_model(model_name, epoch=0):
    model.save(os.path.join(LEARNING_DIR, f'{model_name}_{epoch}_model.h5'))

def save_weights(model_name, epoch=0):
    model.save_weights(os.path.join(LEARNING_DIR, f'{model_name}_{epoch}_weights.h5'))


# Функция для загрузки директорий данных и масок выборок в файл

def load_parameters(model_name):
    parameters = []
    with open(os.path.join(LEARNING_DIR, model_name+'_parameters.txt'), 'r') as file:
        for i, line in enumerate(file):
            # считываем orig_dir_np, eff_dir_np, train_mask, val_mask, test_mask
            if i != 0 and i % 2 == 0:
                if i <= 4: parameters.append(np.array(line.split()))
                else: parameters.append(np.array(line.split()).astype(bool))
    return parameters


# Функции для загрузки весов и полной модели

def load_weights(model_name, epoch=LAST_EPOCH):
    model.load_weights(os.path.join(LEARNING_DIR, f'{model.name}_{epoch}_weights.h5'))
    return model

def load_full_model(model_name, epoch=LAST_EPOCH):
    model = load_model(os.path.join(LEARNING_DIR, f'{model.name}_{epoch}_model.h5'))
    return model

#Сегментация плевральных выпотов

In [133]:
# Определим количество частей на срезе и их разрешение при упрощённой и полной сегментации 
chunks_count = 16 if SIMPLE_3D_f else 1
chunk_res = int(IMG_RESOLUTION/(chunks_count**0.5)) if SIMPLE_3D_f else IMG_RESOLUTION

##Нейронка для сегментации

In [134]:
#@title Самописная функция ошибки и метрики Dice

def dice_coef(y_true, y_pred, shape=1e-6):
    y_true = K.cast(y_true, dtype='float32') # приводим y_true к типу float32
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    # print(f'y_true= {type(y_true)} y_pred= {type(y_pred)}')
    intersection = K.sum(y_true_f * y_pred_f)
    calc1 = (2. * intersection + shape)
    calc2 =  (K.sum(y_true_f) + K.sum(y_pred_f) + shape)
    res = calc1 / calc2
    return  res

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

#get_custom_objects().update({"dice_coef": dice_coef, "dice_coef_loss": dice_coef_loss})

In [135]:
#@title U-Net 3D Model

def conv3d_block(input_tensor, num_filters):
    x = Conv3D(num_filters, 3, padding="same")(input_tensor)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv3D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input_tensor, num_filters):
    e_out = conv3d_block(input_tensor, num_filters)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2))(e_out)
    return e_out, x

def decoder_block(input_tensor, skip_tensor, num_filters):
    x = Conv3DTranspose(num_filters, kernel_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(input_tensor)
    x = concatenate([x, skip_tensor], axis=-1)
    x = conv3d_block(x, num_filters)
    return x

def unet_3d(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    e_out1, e1 = encoder_block(inputs, 64)
    e_out2, e2 = encoder_block(e1, 128)
    e_out3, e3 = encoder_block(e2, 256)
    e_out4, e4 = encoder_block(e3, 512)

    # Bridge
    b1 = conv3d_block(e4, 1024)

    # Decoder
    d1 = decoder_block(b1, e_out4, 512)
    d2 = decoder_block(d1, e_out3, 256)
    d3 = decoder_block(d2, e_out2, 128)
    d4 = decoder_block(d3, e_out1, 64)

    # Output
    outputs = Conv3D(num_classes, 1, activation="sigmoid", padding="same")(d4)

    model = Model(inputs, outputs, name = 'unet_3d')
    return model

In [136]:
#@title VoxResNet 3D Model

def vox_resnet(input_shape, classes, n=[3, 4, 6, 3], k=32):

    def conv(filters):
        return Conv3D(filters, kernel_size=3, strides=1, padding='same',
                      kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1.e-4))

    def deconv(filters):
        return Conv3DTranspose(filters, kernel_size=3, strides=2, padding='same',
                               kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1.e-4))

    def residual_block(x, filters):
        skip = x
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = conv(filters)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = conv(filters)(x)
        x = Add()([x, skip])
        return x

    def upsampling_block(x, filters):
        x = deconv(filters)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x

    def downsample_block(x, filters):
        x = conv(filters)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling3D(pool_size=(2, 2, 2))(x)
        return x

    inputs = Input(shape=input_shape)
    x = inputs

    for i in range(n[0]):
        x = residual_block(x, k)
    x = downsample_block(x, k*2)

    for i in range(n[1]):
        x = residual_block(x, k*2)
    x = downsample_block(x, k*4)

    for i in range(n[2]):
        x = residual_block(x, k*4)
    x = upsampling_block(x, k*2)

    for i in range(n[3]):
        x = residual_block(x, k*2)
    x = upsampling_block(x, k)

    x = residual_block(x, k)
    outputs = Conv3D(classes, kernel_size=3, strides=1, padding='same',
                     kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(1.e-4), activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs, name = 'voxresnet')
    return model

In [137]:
#@title Создание и компиляция модели

input_shape = (chunk_res, chunk_res, MAX_SLICES, 1)
optimizer=Adam(learning_rate=0.0005)

if MODEL == 0:
    model = unet_3d(input_shape, CLASS_COUNT)
else:
    model = vox_resnet(input_shape, CLASS_COUNT)

model.compile(optimizer=optimizer,
              loss=dice_coef_loss,
              metrics=[[BinaryIoU(target_class_ids=[0, 1], threshold=0.9, name='binary_io_u')], dice_coef])

In [138]:
#@title Смотрим характеристики модели

#model_unet_3d.summary()

# Выводим схему модели

#plot_model(model_unet_3d)

In [139]:
#@title Вывод логов обучения

def print_log(current,                           # номер текущего батча
              amount,                            # число всех батчей 
              params):                           # словарь дополнительных параметров для вывода 
  
  bar_len = 20                                   # длина бара 
  percent = int(current * bar_len / amount)      # процент выполненной работы
  progressbar = ''

  for i in range(bar_len):                       # Проходим по всем элементам прогрессбара и добавляем символы в соответствии с прогрессом 
    if(i < percent):
      progressbar += '='
    elif(i == percent):
      progressbar += '>'
    else:
      progressbar += '-'

  # Добавляем в финальное сообщение символ переноса каретки консоли на начальную строку,
  # добавляем информацию о номере батча, количестве всех батчей, прогрессбар.
  # Символ переноса каретки \r добавляется для того, чтобы каждый новый батч перезаписывать
  # вывод - таким образом он не будет засоряться повторяющейся информацией.
  message = "\r" + str(current) + '/' + str(amount) + ' [' + progressbar + ']  ' 
 
  # Добавляем дополнительные параметры в вывод
  for key in params: message += key + ': ' + str(round(params[key], 4)) + '. '
  
  print(message, end='')

In [140]:
#@title Коллбэки

def callbacks(model, model_name, epoch, params, callback, optimizer = optimizer,
              min_delta=1e-3, min_lr=1e-6, max_lr=1e-4, es_patience=8, lr_patience=5, lr_factor=0.7):

    def model_checkpoint():
        if accuracy > callback['mc_accuracy'] and loss < callback['mc_loss']:
            print(f"В сравнении с эпохой {callback['mc_epoch']+1}: {add_name}loss уменьшилось", end=' ')
            print(f"с {round(callback['mc_loss'],4)} до {loss}, а {add_name}accuracy увеличилось с", end=' ')
            print(f"{round(callback['mc_accuracy'],4)} до {accuracy}.")
            callback['mc_loss'] = loss
            callback['mc_accuracy'] = accuracy
            callback['mc_epoch'] = epoch
            model.save_weights(os.path.join(GDRIVE_PATH, DRIVE_DATA_DIR, model_name+'_weights.h5'))
            model.save(os.path.join(GDRIVE_PATH, DRIVE_DATA_DIR, model_name+'_model.h5'))
            print(f'Модель {model_name}.h5 сохранена на эпохе {epoch+1}.')

    add_name = 'val_' if VALIDATION_f else ''
    loss = round(params[add_name+'loss'][epoch], 4)
    accuracy = round(params[add_name+'accuracy'][epoch], 4)

    if epoch > 0:

        if VALIDATION_f:

            es_count = callback['es_count']
            lr_count = callback['lr_count']

            # ModelCheckpoint
            model_checkpoint()

            # ReduceLRonPlateau
            if (callback['lr_loss'] - loss) < min_delta:
                lr_count += 1
                if lr_count >= lr_patience:                   
                    lr_val = round(optimizer.lr.numpy()*lr_factor, 7)
                    a = 'а' if lr_count in [2,3,4] else ''
                    print(f"Начиная с {callback['lr_epoch']+1}-й эпохи val_loss особо не уменьшалось {lr_count} раз{a} подряд.")
                    if lr_val < min_lr:
                        optimizer.learning_rate.assign(max_lr)
                        print(f'Learning rate стал ниже своего минимального значения и вновь восстановлен до {max_lr}.')
                    else:
                        optimizer.learning_rate.assign(lr_val)
                        print(f'Learning rate уменьшается до {lr_val}.')                 
                    callback['lr_loss'] = loss
                    callback['lr_epoch'] = epoch
                    lr_count = 0
            else:
                callback['lr_loss'] = loss
                callback['lr_epoch'] = epoch
                lr_count = 0
            callback['lr_count'] = lr_count
      
            # EarlyStopping
            if (callback['es_loss'] - loss) < min_delta:
                es_count += 1
                if es_count >= es_patience:
                    print(f"Угроза переобучения! Восстанавливаются веса модели лучшей эпохи: {callback['mc_epoch']+1}.")
                    model.load_weights(os.path.join(GDRIVE_PATH, DRIVE_DATA_DIR, model_name+'_weights.h5'))
                    print(f'Веса модели {model_name} восстановлены. Дальнейшее обучение приостановлено.')
                    es_count = -1
            else:
                es_count = 0
            callback['es_count'] = es_count

        else:
            model_checkpoint()
    else:
        callback.update({
            'lr_loss': loss,
            'es_loss': loss,
            'mc_loss': loss,
            'mc_accuracy': accuracy
        })
        model.save_weights(os.path.join(GDRIVE_PATH, DRIVE_DATA_DIR, model_name+'_weights.h5'))

    return model, callback

In [141]:
#@title Загрузка модели при необходимости
if ADDITIONAL_TRAIN_f:
    orig_dir_np, eff_dir_np, train_mask, val_mask, test_mask = load_parameters(model.name)
    model = load_weights(model.name)
    # model = load_full_model(model.name, LAST_EPOCH)

##Получение массивов директорий

In [142]:
#@title Массивы директорий экземпляров

# При первом обучении 
if not ADDITIONAL_TRAIN_f:
    # Получаем списки папок LUNG1-XXX и переводим их в numpy массив
    orig_dir_np = np.array(sorted(get_folders(original_dir))) # папки с оригинальными изображениями
    eff_dir_np = np.array(sorted(get_folders(effusions_dir))) # папки с плевральными выпотами

    # Отфильтровываем директории в eff_dir_np, которых нет в orig_dir_np
    eff_dir_np = eff_dir_np[np.in1d(eff_dir_np, orig_dir_np)]

print(f'Все образцы находятся в следующих папках:  {orig_dir_np[:5]}..')
print(f'Маски выпотов хранятся в следующих папках: {eff_dir_np[:5]}..')
print('Всего экземпляров с выпотами:', len(eff_dir_np))
print('Общее количество экземпляров:', len(orig_dir_np))

Все образцы находятся в следующих папках:  ['LUNG1-302' 'LUNG1-303' 'LUNG1-304' 'LUNG1-305' 'LUNG1-306']..
Маски выпотов хранятся в следующих папках: ['LUNG1-302' 'LUNG1-303' 'LUNG1-304' 'LUNG1-305' 'LUNG1-306']..
Всего экземпляров с выпотами: 121
Общее количество экземпляров: 121


In [143]:
#@title Уравнивание классов при необходимости

# Установка начального «семени» в случайные значения
np.random.seed()

if CLASS_EQUATING_f and not ADDITIONAL_TRAIN_f:

    # Получаем директории экземпляров, свободных от выпотов
    no_eff_dir_np = np.setdiff1d(orig_dir_np, eff_dir_np)
    print('Без выпотов до "обработки":', len(no_eff_dir_np), no_eff_dir_np[:5])

    # Перемешиваем массив
    np.random.shuffle(no_eff_dir_np)

    # Рубаем массив по количеству экземпляров с выпотами, объединяем с выпотами и сортируем
    orig_dir_np = np.sort(np.concatenate((no_eff_dir_np[:len(eff_dir_np)], eff_dir_np)))
    print('Общий после "обработки" (выпоты x2):', len(orig_dir_np), orig_dir_np[:5])

Без выпотов до "обработки": 0 []
Общий после "обработки" (выпоты x2): 121 ['LUNG1-302' 'LUNG1-303' 'LUNG1-304' 'LUNG1-305' 'LUNG1-306']


##Формирование train, val, test масок и массива выходных данных

После проведённого ряда экспериментов, связанных с ограничением памяти по формированию входного массива, принято решение формировать входные данные партиями и постепенно обучать модель.

Сформируем метки наличия выпотов `effusion_labels` для экземпляров (директорий) оригинальных 3D изображений путём сравнения наличия директорий `LUNG1-XXX` в папках `ORIGINAL_DIR` и `EFFUSIONS_DIR`.

Значения `effusion_labels`: 1 - выпот имеется; 0 - без выпота.

In [144]:
# Создаём список меток через разницу папок в виде булевых значений
effusion_labels = np.isin(orig_dir_np, eff_dir_np).astype(bool)

# Количество 3D-изображений лёгких (директорий LUNG1-XXX) - объём датасета 
orig_elements_count = len(orig_dir_np)

# Результат
print(orig_dir_np[:5])
print(eff_dir_np[:5])
print(effusion_labels[:5])
print(f'Всего лёгких с выпотом {np.sum(effusion_labels==1)} из {orig_elements_count} экземпляров.')

['LUNG1-302' 'LUNG1-303' 'LUNG1-304' 'LUNG1-305' 'LUNG1-306']
['LUNG1-302' 'LUNG1-303' 'LUNG1-304' 'LUNG1-305' 'LUNG1-306']
[ True  True  True  True  True]
Всего лёгких с выпотом 121 из 121 экземпляров.


Получим маски для обучающей, проверочной и тестовой выборок на весь объём данных с пропорциональным разделением на лёгкие с плевральным выпотом и без него для каждого набора.

In [145]:
# Функция для вывода результата разделения выборок

def split_results(sampling_name, sampling, mask, idxs):

    print(f'{sampling_name} выборка имеет форму {sampling.shape}')
    print(f'Занимает от общей выборки: {round(len(idxs)/len(effusion_labels)*100, 2)}%\n')

    # Проверяем, что сумма элементов маски равна количеству индексов в выборке 
    assert mask.sum() == len(idxs)

In [146]:
if not ADDITIONAL_TRAIN_f:

    # Соотношение выборок train:val = 4:1, вычисляется из размера тестовой выборки
    test_size = 10
    val_size = (100 - test_size) / 5 if VALIDATION_f else 0
    train_size = 100 - test_size - val_size

    # Процентное соотношение выборок
    sampling_ratio = (train_size/100, val_size/100, test_size/100)
    sampling_ratio

In [147]:
if not ADDITIONAL_TRAIN_f:

    # Получаем индексы единиц и нулей в выходном массиве
    ones_idxs = np.where(effusion_labels == 1)[0]
    zeros_idxs = np.where(effusion_labels == 0)[0]

    # Перемешиваем индексы
    np.random.shuffle(ones_idxs)
    np.random.shuffle(zeros_idxs)

    # Делим массив индексов единиц на 3 части
    ones_idxs_train, ones_idxs_val, ones_idxs_test = np.split(ones_idxs,
                                                              [int(len(ones_idxs)*
                                                                  sampling_ratio[0]),
                                                              int(len(ones_idxs)*
                                                                  (sampling_ratio[0]+
                                                                    sampling_ratio[1]))])

    # Делим массив нулей на 3 части
    zeros_idxs_train, zeros_idxs_val, zeros_idxs_test = np.split(zeros_idxs,
                                                                [int(len(zeros_idxs)*
                                                                      sampling_ratio[0]),
                                                                  int(len(zeros_idxs)*
                                                                      (sampling_ratio[0]+
                                                                      sampling_ratio[1]))])

    # Объединяем индексы из предыдущих шагов
    train_idxs = np.concatenate([ones_idxs_train, zeros_idxs_train])
    val_idxs = np.concatenate([ones_idxs_val, zeros_idxs_val])
    test_idxs = np.concatenate([ones_idxs_test, zeros_idxs_test])

    # Создаём маску, которая будет соответствовать номерам элементов входного массива
    train_mask = np.zeros((orig_elements_count,), dtype=bool)
    train_mask[train_idxs] = True

    val_mask = np.zeros((orig_elements_count,), dtype=bool)
    val_mask[val_idxs] = True

    test_mask = np.zeros((orig_elements_count,), dtype=bool)
    test_mask[test_idxs] = True

    # Сохраняем маски в файл NetParameters.txt для отслеживания различных результатов
    save_parameters(model.name, train_mask, val_mask, test_mask)

    # Проверяем полученную форму масок
    assert train_mask.shape[0] == orig_elements_count

## Формирование списков батчей из датасета для train, val и test выборок

Сформируем полные выборки директорий и меток для 3D изображений.

In [148]:
# Получаем train, val и test директории и метки выборок
x_train, y_train = orig_dir_np[train_mask], effusion_labels[train_mask]
x_val, y_val = orig_dir_np[val_mask], effusion_labels[val_mask]
x_test, y_test = orig_dir_np[test_mask], effusion_labels[test_mask]

if not ADDITIONAL_TRAIN_f:
    # Выводим результаты
    split_results('Тренировочная ', y_train, train_mask, train_idxs)
    split_results('Проверочная ', y_val, val_mask, val_idxs)
    split_results('Тестовая ', y_test, test_mask, test_idxs)

Тренировочная  выборка имеет форму (87,)
Занимает от общей выборки: 71.9%

Проверочная  выборка имеет форму (21,)
Занимает от общей выборки: 17.36%

Тестовая  выборка имеет форму (13,)
Занимает от общей выборки: 10.74%



Разобьём датасет на батчи:

- зададим параметры шаг смещения `SIZE_STEP` и размер батча `BATCH_SIZE`, подготовим пустой список.

- рассчитаем, на сколько батчей `steps` возможно разбить список директорий изображений при заданном шаге смещения. 

- в цикле возьмём нужные значения из общего списка и поместим их в батч.

In [149]:
# Функция формирования батча списка названий директорий с изображениями и списка меток

def get_batch_lists(dir_list,         # список директорий с изображениями для формирования батчей
                    labels,           # метки 3D изображений
                    batch_size,       # размер батча
                    size_step):       # шаг смещения

    steps = len(dir_list)//size_step  # кол-во батчей в списке при параметрах выше
    list_batch_ID = []                # список для батча директорий с изображениями
    list_batch_labels = []            # список для батча меток, соответствующих 3D изображениям

    for step in range(steps):
        
        # Берём элементы с нужным индексом и присоединяем их к соответствующим спискам
        batch_ID = dir_list[step*size_step:step*size_step + batch_size]
        list_batch_ID.append(batch_ID)
        batch_lbl = labels[step*size_step:step*size_step + batch_size]
        list_batch_labels.append(batch_lbl)

    return [list_batch_ID, list_batch_labels]

При заданных значениях шага смещения `SIZE_STEP` и батча  `BATCH_SIZE` сформируем список списков батчей директорий изображений и их меток для `train`, `val` и `test` выборок.

Шаг смещения < размер батча можно использовать в случае аугментации изображений.

In [150]:
# Формируем наборы батчей для train, val и test выборок
batches_train = get_batch_lists(x_train, y_train, BATCH_SIZE, SIZE_STEP)
batches_val = get_batch_lists(x_val, y_val, BATCH_SIZE, SIZE_STEP)
batches_test = get_batch_lists(x_test, y_test, BATCH_SIZE, SIZE_STEP)

# Смотрим что получилось
step = 1
print('Содержимое батча директорий изображений:', batches_train[0][step])
print('Содержимое батча меток:', batches_train[1][step], '\n')
print('Размер батча директорий изображений:', len(batches_train[0][step]))
print('Размер батча меток:', len(batches_train[1][step]))

Содержимое батча директорий изображений: ['LUNG1-304']
Содержимое батча меток: [ True] 

Размер батча директорий изображений: 1
Размер батча меток: 1


In [151]:
#@title Проверка сформированных батчей
def check_batches(batches):
    check_flags = []
    for i, dir in enumerate(batches[0]):
        if dir in orig_dir_np:
            idx = np.where(orig_dir_np == dir)
            if batches[1][i] == effusion_labels[idx]:
                check_flags.append(True)
            else: check_flags.append(False)
    return check_flags

train_check = check_batches(batches_train)
val_check = check_batches(batches_val)
test_check = check_batches(batches_test)

check_count = 0
check_values = True
for check in (train_check, val_check, test_check):
    check_values = check_values and all(np.unique(check))
    check_count += len(check)

print(check_values and (check_count == len(orig_dir_np)))

True


In [152]:
#@title Функция для получения картинки в представлении numpy

def get_img_np(image_path, resolution = IMG_RESOLUTION):
    
    depth_pool = () 
    if image_path.endswith('dcm'):
        image_np = dicom.dcmread(image_path).pixel_array.astype('int16')
    elif image_path.endswith('png'):
        image_np = np.array(Image.open(image_path).convert('L'))
    else:         # endswith('nii.gz')
        image_np = nib.load(image_path).get_fdata().astype('int16')
        depth_pool = (1,)

    # Меняем разрешение, если требуется
    if resolution != 512:
        pool_size = 1/(512/resolution)
        image_np = zoom(image_np, (pool_size, pool_size) + depth_pool, mode='nearest')

    return image_np

In [153]:
#@title Посмотрим, что получилось на батче на шаге step

img_res = 128

def get_img_path(dir, i):
    img_dir = get_full_path(os.path.join(dir, batches_train[0][step][i]))               # путь к папке со срезами
    img_path = os.path.join(img_dir,
                            sorted(os.listdir(img_dir))[len(os.listdir(img_dir)) // 2]) # средний срез
    return img_path

# Сформируем numpy для изображения и маски
for i in range(BATCH_SIZE):

    img = get_img_np(get_img_path(original_dir, i), img_res)

    if batches_train[1][step][i]: # если есть выпот - получаем его маску
        img_path = get_img_path(effusions_dir, i)
        mask = get_img_np(img_path, img_res)
        if img_path.endswith('.gz'): mask = mask[:, :, mask.shape[2] - mask.shape[2] // 2 - 1]
        mask_name = 'Есть маска'
    else:                          # если нет - формируем пустое изображение
        mask = np.zeros((img_res, img_res))
        mask_name = 'Нет маски'

    plot_pics((img, mask), ('Оригинальное изображение', 'Маска выпота'))

IndexError: list index out of range

##Генерация батчей с данными

Добавим недостающие директории в датасет с выпотами для равного количества оригинальных и сегментированных экземпляров.

In [154]:
for dir in np.setdiff1d(orig_dir_np, eff_dir_np):
    if not os.path.exists(os.path.join(effusions_dir, dir)):
        os.makedirs(os.path.join(effusions_dir, dir))

In [155]:
#@title Дополнительные функции

# Функции преобразования сегментированного изображения в метки классов и обратно при 2D или упрощённой 3D сегментации
def bool_to_labels(image, lung_folder): # 2D или 3D картинка для преобразования

    class_lbl = np.where(eff_dir_np == lung_folder)[0]
    image[np.where(np.all(image == 0))] = class_lbl * 2
    image[np.where(np.all(image == 1))] = class_lbl * 2 + 1
  
    return image


def labels_to_bool(image):

    class_lbl = np.unique(image)[0]//2    
    image[np.where(np.all(image == class_lbl * 2))] = 0
    image[np.where(np.all(image == class_lbl * 2 + 1))] = 1

    return image

  
# Функция ограничения значений пикселей в предполагаемых пределах выпота для повышения точности

def apply_bounds(image_3d, image):

    image_hu = image_3d * np.int16(image.RescaleSlope) + np.int16(image.RescaleIntercept)
    image_hu[image_hu>MAX_BOUND] = MAX_BOUND
    image_hu[image_hu<MIN_BOUND] = MIN_BOUND
    image_3d = ((image_hu - np.int16(image.RescaleIntercept))/np.int16(image.RescaleSlope)).astype(np.int16)

    return image_3d


# Функция определения позиции окна при разделении 3D изображения на части

def get_chunk_position(num, # номер обрабатываемой части
                       n):  # количество частей
    row = int(num // np.sqrt(n))
    col = int(num % np.sqrt(n))
    return (row, col)

In [156]:
#@title Генератор батчей

# Функция для получения numpy массива 3D изображений лёгкого

def get_3d_image(path_dir,                    # путь к датасету (оригинальные/маски)
                 lungfolder,                  # название папки экземпляра
                 chunks_count,                # количество срезов в 3D изображении
                 simple3d_batch_num,          # номер батча в случае упрощённой 3D сегментации
                 chunk_res):

        image_3d_np = np.zeros((chunk_res, chunk_res, MAX_SLICES),    # пустой 3D массив
                               dtype = np.int16)

        chunk_row, chunk_col = get_chunk_position(simple3d_batch_num, chunks_count)
        full_path = get_full_path(os.path.join(path_dir, lungfolder))           # путь к папке с изображениями
        image_files = sorted(os.listdir(full_path))                             # изображения в папке экземпляра
        images_count = len(image_files)                                         # количество изображений в папке экземпляра
        
        # Если каталог не пуст (создан при распаковке, не программно)
        if images_count > 0:


#доработать части


            if image_files[0].endswith('.gz'):
                # получаем 3D массив маски экземпляра
                image_path = os.path.join(full_path, image_files[0])
                nii_img = get_img_np(image_path)
                # поворачиваем его против часовой, ставим обратный порядок срезов и вставляем в image_3d_np
                image_3d_np[:, :, :nii_img.shape[2]] = np.rot90(nii_img)[:, :, ::-1]

            else:           # endswith '.dcm' or '.png'                  
                # для всех файлов в каталоге по указанному пути:
                for slice_num in range(images_count):
                    # читаем 2D картинку и берём из неё нужную часть
                    image_path = os.path.join(full_path, image_files[slice_num])
                    img_np = get_img_np(image_path)[chunk_row * chunk_res:(chunk_row+1) * chunk_res,
                                                    chunk_col * chunk_res:(chunk_col+1) * chunk_res]
                    # добавляем её в 3D массив
                    image_3d_np[:, :, slice_num] = img_np

            # Для оригинальных (DICOM) изображений применяем границы пикселей по HU
            if image_path.endswith('dcm'):
                image_3d_np = apply_bounds(image_3d_np, dicom.dcmread(image_path))

            # В случае 2D или упрощённой 3D сегментации - переводим булевую маску в метки экземпляров
            #if SIMPLE_3D_f and image_path.endswith('png'): image_3d_np = bool_to_labels(image_3d_np, lungfolder)
        
        return image_3d_np


def batch_generator(step,                                   # шаг обучения
                    set_batches,                            # список из списков батчей директорий с изображениями и списка меток
                    chunks_count = 1,                       # количество частей, на которые поделён 2D срез 3D изображения
                    batch_size = BATCH_SIZE,                # размер батча для генератора
                    simple3d_batch_num = 0,                 # номер батча в случае упрощённой 3D сегментации
                    chunk_res = IMG_RESOLUTION):

    # Создаём пустые 5D-массивы
    x_batch_np = np.zeros((batch_size, chunk_res, chunk_res, MAX_SLICES, 1), dtype = np.int16)
    y_batch_np = np.zeros((batch_size, chunk_res, chunk_res, MAX_SLICES, 1), dtype = np.int16)

    # Для всех директорий в каталоге по указанному пути:
    for i, lung_folder in enumerate(set_batches[0][step]):   # список батча директорий с изображениями

        # Записываем 3D-массив экземпляра в нужный элемент 5D массива x_batch_np
        np.copyto(x_batch_np[i, :, :, :, 0], get_3d_image(original_dir, lung_folder, chunks_count, simple3d_batch_num, chunk_res))

        # Если есть выпот - закидываем картинки в 3D изображение и вставляем его в 5D массив y_batch_np
        if set_batches[1][step][i]: # метка выпота
            np.copyto(y_batch_np[i, :, :, :, 0], get_3d_image(effusions_dir, lung_folder, chunks_count, simple3d_batch_num, chunk_res))

    # Нормализуем данные батча оригинальных изображений к значениям от 0 до 1
    x_batch_np = (x_batch_np.astype(np.float16) - np.float16(MIN_BOUND)) / (np.float16(MAX_BOUND) - np.float16(MIN_BOUND))

    # Возвращаем батч нормализованных данных 3D изображений
    return (x_batch_np, y_batch_np)

###Проверим работу генератора

In [157]:
# Отметка текущего времени
cur_time = datetime.datetime.now()

# Формируем батч тренировочной выборки
step = 0
if SIMPLE_3D_f:
    batch_step_train = batch_generator(step, batches_train,
                                       chunks_count, BATCH_SIZE)
else:
    batch_step_train = batch_generator(step, batches_train)

print(batch_step_train[0].dtype, batch_step_train[0].shape, batch_step_train[1].shape)

# Получаем время обработки
time_diff = datetime.datetime.now() - cur_time

print('Изображения Оригинальные тренировочные загружены.')
print(f'Время загрузки: {(time_diff.seconds//60)%60} мин {time_diff.seconds%60} с')
print('Директории:', batches_train[0][step])
print('Метки выпота:', batches_train[1][step])

float16 (1, 16, 16, 304, 1) (1, 16, 16, 304, 1)
Изображения Оригинальные тренировочные загружены.
Время загрузки: 0 мин 0 с
Директории: ['LUNG1-302']
Метки выпота: [ True]


Посмотрим картинки из батча (с выпотом и без). Первые две - оригинальные изображения, вторые - маски к ним.

Для этого в полученных на предыдущем шаге метках должны присутствовать False и True и BATCH_SIZE должен быть больше 1. Если их нет - необходимо изменить параметр step.

In [158]:
if BATCH_SIZE > 1 and len(np.unique(batches_train[1][step])) > 1:
    print('IMG_RESOLUTION:', IMG_RESOLUTION)

    slice_num = 64  # Номер среза
    idx = []        # Индексы экземпляра с выпотом idx[0] и без idx[1]

    idx.append(np.where(batches_train[1][step])[0])
    idx.append(np.where(~batches_train[1][step])[0])
    images_name = ('Оригинальные изображения', 'Маски')

    for i in range(2):

        # Получаем данные изображений
        img1_np = batch_step_train[i][idx[0][0],:,:,slice_num].squeeze(axis=2)
        img2_np = batch_step_train[i][idx[1][0],:,:,slice_num].squeeze(axis=2)

        # Выводим изображения
        print(images_name[i])
        plot_pics((img1_np, img2_np), ('С выпотом', 'Без выпота'))

На выходе мы имеем готовый батч в виде массивов оригинальных и сегментированных 3D изображений для подачи на обучение.

Формы массивов (None, 512, 512, 304, 1) (для полной 3D сегментации):

(размер батча, ширина картинки, высота картинки, количество срезов, количество каналов).



##Обучение Train on batch

In [159]:
#@title Train Eval step

def train_eval_step(step, step_data, steps, params, mode_str, int_start, batch_num = 0):

    # создаём пустые списки для сбора данных по шагам внутри эпохи
    loss = []
    biou = []
    dice = []
    #acc = []
    
    # присваиваем наименования ключам в словаре
    loss_str = mode_str + 'loss'
    biou_str = mode_str + 'BinaryIoU'
    dice_str = mode_str + 'dice'
    #accuracy_str = mode_str + 'accuracy'

    # train mode
    if mode_str == '':
        # пропускаем проверочный батч данных через функцию оценки модели,
        # получаем ошибку и точность как список
        result_step = model.train_on_batch(step_data[0], step_data[1])
        # задаём параметры в словаре, которые будем выводить
        time_diff = datetime.datetime.now() - int_start     # получаем время обработки
        params = {
            'Время(сек.) на эпохе': time_diff.seconds,      # считаем время обучения на данной эпохе и добавляем в словарь
            loss_str: round(result_step[0], 4),             # добавляем в словарь ошибку на шаге обучения
            biou_str: round(result_step[1], 4),
            dice_str: round(result_step[2], 4)
        }
        # печатаем отдельной функцией текущие данные на шаге обучения
        print_log(step * chunks_count + batch_num, steps * chunks_count, params)

    # evaluate mode
    else:
        # оцениваем модель на проверочной базе
        result_step = model.evaluate(step_data[0], step_data[1], verbose=0)

    # собираем ошибку и точность на шаге
    loss.append(result_step[0])
    biou.append(result_step[1])
    dice.append(result_step[2])

    return model, params, loss, biou, dice

In [160]:
#@title Train Eval mode
def train_eval_mode(model,                  # модель
                    steps,                  # количество шагов на эпохе
                    batches,                # батчи директорий и меток
                    loss_lrn,               # список для сбора усреднённых ошибок в конце эпохи
                    biou_lrn,
                    dice_lrn,
                    int_start=None,         # начальное время эпохи
                    params=None,            # словарь параметров эпохи
                    mode_str=''):           # режим обучения ('') и проверки ('val_')

    # присваиваем наименования ключам в словаре
    loss_str = mode_str + 'loss'
    biou_str = mode_str + 'binary_io_u'
    dice_str = mode_str + 'dice'
    #accuracy_str = mode_str + 'accuracy'

    for step in range(steps):   
        if SIMPLE_3D_f:
            step_data_2d = []               # данные в формате (slices_count, img_res, img_res, 1)
            for batch_num in range(chunks_count):
                step_data = batch_generator(step, batches, chunks_count, 1, batch_num, chunk_res)
                #print(step, batch_num, step_data[0].shape, step_data[1].shape)
                '''for i in range(2):
                    # получаем массив оригинального/сегментированного изображения
                    step_data_2d.append(batch3d_to2d(step_data[i]))
                    ниже это выше batch3d_to2d
                    # Убираем единичные оси и конвертируем массив под формат данных для 2D сети
                    array = np.transpose(np.squeeze(array), (2, 0, 1))
                    
                    
                    # создаём массив предсказаний модели на основе срезов 2D изображений
                    predictions = np.zeros((MAX_SLICES, IMG_RESOLUTION, IMG_RESOLUTION, 1), dtype = np.int16)
                    # каждый срез в 3d массиве
                    for slice_num in range(step_data_2d[i].shape[0]):
                        # пропускаем через обученную 2D модель и получаем предсказание
                        # ********************************* model_2d
                        prediction_2d = model.predict(step_data_2d[i][slice_num])
                        x_batch_np = np.zeros((batch_size, IMG_RESOLUTION, IMG_RESOLUTION, slices_count, 1), dtype = np.int16)
                        np.copyto(y_batch_np[i, :, :, :, 0], get_3d_image(effusions_dir, lung_folder, slices_count, simple3d_batch_num))
                    for i in range(original_volume.shape[0]):

                        # сохранить предсказание в список
                        predictions.append(prediction_2d)

                        # объединить предсказания для каждого среза в одно 3D изображение
                        prediction_volume = np.concatenate(predictions, axis=0)

                        # объединить полученное 3D изображение с оригинальным 3D изображением и сегментированным 3D изображением
                        merged_volume = np.stack([original_volume, segmented_volume, prediction_volume], axis=3)

                        # использовать полученные данные для обучения 3D модели'''



                model, params, loss, biou, dice = train_eval_step(step, step_data, steps, params, mode_str, int_start, batch_num)
        else:
            # получаем батч данных на этапе
            step_data = batch_generator(step, batches)
            model, params, loss, biou, dice = train_eval_step(step, step_data, steps, params, mode_str, int_start)

    # собираем усреднённые ошибку и точность от всех шагов на эпохе
    loss_lrn.append(np.mean(loss))
    biou_lrn.append(np.mean(biou))
    dice_lrn.append(np.mean(dice))

    # запоминаем и округляем последнее записанное усреднённое значение для отображения по ходу обучения
    # в train mode перезаписываем значения в словаре, в evaluate mode добавляем данные в словарь
    params.update({
        loss_str: round(loss_lrn[-1],4),
        biou_str: round(biou_lrn[-1],4),
        dice_str: round(dice_lrn[-1],4)
    })

    return model, (loss_lrn, biou_lrn, dice_lrn, params, model)

In [161]:
#@title Model train on batch

def model_train_on_batch(model, optimizer=optimizer, epochs=50, last_epoch=0):

    # создаём пустые списки для сбора усредненных данных от шагов в конце эпохи
    loss_train_lrn =[]
    biou_train_lrn = []
    dice_train_lrn = []
    loss_val_lrn =[]
    biou_val_lrn = []
    dice_val_lrn = []

    # определяем количество шагов
    steps_train = len(batches_train[0])
    steps_val = len(batches_val[0])

    callback = {
        'mc_loss': 100, 'mc_accuracy': 0, 'mc_epoch': 0,
        'lr_count': 0, 'lr_epoch': 0, 'lr_loss': 0,
        'es_count': 0, 'es_epoch': 0, 'es_loss': 0,
    }

    # запоминаем время старта обучения
    # start_time = datetime.datetime.now()

    # запускаем цикл обучения по эпохам
    for epoch in range(epochs):
        
        # выводим текущую эпоху и общее число эпох
        print('Эпоха', last_epoch+epoch+1, '/', last_epoch+epochs)

        # запускаем цикл обучения по шагам внутри эпохи
        model, train_params = train_eval_mode(model, steps_train, batches_train,
                                              loss_train_lrn, biou_train_lrn, dice_train_lrn,
                                              datetime.datetime.now())

        if VALIDATION_f:
            # оцениваем модель на проверочной базе
            model, val_params = train_eval_mode(model, steps_val, batches_val,
                                                loss_val_lrn, biou_val_lrn, dice_val_lrn,
                                                params=train_params[3], mode_str='val_')

        # печатаем отдельной функцией усреднённые данные в конце текущей эпохи обучения
        print_params = val_params[3] if VALIDATION_f else train_params[3]
        print_log(steps_train * chunks_count, steps_train * chunks_count, print_params)

        # Вручную переносим каретку на следующую строку,
        # чтобы не стирать финальные значения сети на эпохе
        print()

        model_history = {
            'loss': train_params[0],
            'binary_io_u': train_params[1],
            'dice': train_params[2]
        }
        
        if VALIDATION_f:
            model_history.update({
                'val_loss': val_params[0],
                'val_binary_io_u': train_params[1],
                'dice': train_params[2]
            })

        model.save_weights(os.path.join(GDRIVE_PATH, DRIVE_DATA_DIR, f'{model.name}_{str(last_epoch+epoch+1)}_weights.h5'))
        model.save(os.path.join(GDRIVE_PATH, DRIVE_DATA_DIR, f'{model.name}_{str(last_epoch+epoch+1)}_model.h5'))
        
        '''model, callback = callbacks(model, model_name, epoch, model_history, callback)
        if callback['es_count'] == -1: break'''

    return model, model_history

In [162]:
print('IMG_RESOLUTION =', IMG_RESOLUTION)
print('BATCH_SIZE =', BATCH_SIZE)
print(f'Model: {model.name}')
print('Chunks count & resolution:', chunks_count, chunk_res)

# Обучаем модель на батчах
model, model_h = model_train_on_batch(model, optimizer, last_epoch = LAST_EPOCH)

IMG_RESOLUTION = 16
BATCH_SIZE = 1
Model: unet_3d
Chunks count & resolution: 1 16
Эпоха 1 / 50
11/87 [==>-----------------]  Время(сек.) на эпохе: 117. loss: 1.0. BinaryIoU: 0.4551. dice: 0.0. 42. 

KeyboardInterrupt: 